In [ ]:
pip install keras-tuner --upgrade

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    model = keras.Sequential([
        layers.Conv2D(hp.Int('input_units', min_value=32, max_value=512, step=32), (3,3), activation='relu', input_shape=(224,224,3)),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(hp.Int('dense_units', min_value=64, max_value=512, step=64), activation='relu'),
        layers.Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='vgg16_tuning')

tuner.search_space_summary()

tuner.search(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

tuner.results_summary()
